In [1]:
%pprint

Pretty printing has been turned OFF


In [2]:
from src.core.SDR import *

Create an SDR with a range of 32

In [3]:
sdr = SDR(32)
print("Binary Array: ",sdr.toBinaryArray())
print("Index Array: ",sdr.toIndexArray())
print("Density: " ,sdr.density())
print("Sparsity: " ,sdr.sparsity())

Binary Array:  [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
Index Array:  [4, 7, 9, 13, 19, 21, 28, 30]
Density:  0.25
Sparsity:  0.75


The array above is all the indices of the on bits in the SDR

The sparsity of the SDR above is very low. SDRs work better when they have a greater range and sparsity.

In [4]:
sdr.range = 1024
sdr.random(16) # create new active bits, randomly
print(sdr.toIndexArray())
print("Density: " ,sdr.density())
print("Sparsity: " ,sdr.sparsity())

[63, 107, 190, 200, 318, 332, 429, 487, 554, 614, 647, 758, 773, 860, 911, 961]
Density:  0.015625
Sparsity:  0.984375


### Operations

**Subsample** - Evenly remove active bits from the SDR

In [5]:
print(sdr.subsample(8)) # subsample down to a size of 8 bits

[63, 190, 318, 429, 554, 647, 773, 911]


**Union** - Combine multiple SDRs

In [6]:
sdr.union([SDR().indices,SDR().indices])

[63, 107, 190, 200, 318, 332, 429, 487, 554, 614, 647, 758, 773, 860, 911, 961, 233, 257, 604, 935, 1129, 1535, 1543, 2047, 152, 308, 766, 994, 1074, 1353, 1695, 1948]

**Overlap** - Matches the indices (on bits) that exist in both SDRs

In [7]:
sdr.indices = [63, 190, 318, 429, 554, 647, 773, 924] # reassignment, for ease of demonstration
print(sdr.overlap([63, 113, 318, 417, 554, 647, 784]))

[63, 318, 554, 647]


**Subtract** - Removes the matching indices between the two SDRs (the opposite of overlap)

In [8]:
print(sdr.subtract([63, 113, 318, 417, 554, 647, 784]))

[190, 429, 773, 924]


**Difference** - Combines the SDRs and removes the overlap

In [9]:
print(sdr.difference([63, 113, 318, 417, 554, 647, 784]))

[190, 429, 773, 924, 113, 417, 784]
